In [14]:
from tkinter import *
from  tkinter  import ttk
from tkinter import messagebox as mbox  
from pandastable import Table
import pandas as pd

In [15]:
class Server:
    def __init__(self, data = None, sort_by_col = None, sort_order = None, search_by_col = None, search_content = None):
        self.data = data
        self.sort_by_col = sort_by_col
        self.sort_order = sort_order
        self.search_by_col = search_by_col
        self.search_content = search_content
        
    def my_sort(self, data, sort_by_col, sort_order):
        """
        Sort function.
        Inputs:
        - df: DataFrame
                the data to sort.
        - by_col: str or list of str
                Sort by column(s).
        - sort_type: str or list of str
                asc: ascending, desc: descending

        Returns: DataFrame
        """
        if sort_order == 'Ascending':
            return data.sort_values(sort_by_col, ascending= True)
        elif sort_order == 'Descending':
            return data.sort_values(sort_by_col, ascending= False)
        return -1

    def my_search(self, data, search_by_col, search_content):
        """
        Sort function.
        Inputs:
        - df: DataFrame
                the data to search.
        - by_col: str or list of str
                Search by column(s).
        - text: str or list of str
                content to search

        Returns: DataFrame
        """
        if search_by_col == "Vehicle Type":
            return data[data[search_by_col] == int(search_content)]
        elif search_by_col == "Trip Length":
            # return trip length between the given content is more reasonable
            return data[(data[search_by_col] >= float(search_content[0])) & (data[search_by_col] <= float(search_content[1]))].sort_values(by = search_by_col)
        elif "DirectionTime" in search_by_col:
            time1 = '2019-08-30 '+search_content[0]+':'+search_content[1]+':'+search_content[2]
            time2 = '2019-08-30 '+search_content[3]+':'+search_content[4]+':'+search_content[5]
            return data[(data[search_by_col] >= time1) & (data[search_by_col] <= time2)].sort_values(by = search_by_col)

        return data[data[search_by_col].str.contains(search_content)]

In [16]:
class entry(Entry):
    def __init__(self, master, placeholder,**kw):
        super().__init__(master,**kw)
 
        self.placeholder = placeholder
        self._is_password = True if placeholder == "password" else False
 
        self.bind("<FocusIn>", self.on_focus_in)
        self.bind("<FocusOut>", self.on_focus_out)
 
        self._state = 'placeholder'
        self.insert(0, self.placeholder)
        self.configure(foreground = 'grey')
 
    def on_focus_in(self, event):
        if self._is_password:
            self.configure(show='*')
 
        if self._state == 'placeholder':
            self._state = ''
            self.delete('0', 'end')
        self.configure(foreground = 'black')
    def on_focus_out(self, event):
        if not self.get():
            if self._is_password:
                self.configure(show='')

            self._state = 'placeholder'
            self.insert(0, self.placeholder)
            self.configure(foreground = 'grey')

In [17]:
class interface:
    def __init__(self, data = None, window = None, button_home = None, button_clear = None, button_sort = None, label_sort = None, label_sort_by_col = None, label_sort_order = None, comboxlist_by_col = None, comboxlist_order = None):
        self.data = data
        self.window = window
        self.button_home = button_home
        self.button_clear = button_clear
        self.button_sort = button_sort
        
        self.label_sort = label_sort
        self.label_sort_by_col = label_sort_by_col
        self.label_sort_order = label_sort_order
        
        self.comboxlist_by_col = comboxlist_by_col
        self.comboxlist_order = comboxlist_order

    def init_interface(self):
        self.window = Tk(className=" Inquiry System for Taiwan traffic data")
        self.window.geometry('1300x900+600+200')
        self.window.resizable(width=False, height=False) #禁止改变窗口大小
        self.button_home = Button(self.window,text="Homepage",font=("Arial", 20),command= self.gohome,width=10,height = 1).place(x=1100,y=12)
        self.button_clear = Button(self.window,text="Clear output",font=("Arial", 20),command= self.Reset,width=10,height = 1).place(x=1100,y=72)
        self.button_sort = Button(self.window,text="Sort",font=("Arial", 15),command= self.sort,width=15,height = 1).place(x=900,y=20)
        
        self.label_sort = Label(self.window,text="Sort:",font=("Courier", 20, "bold", "italic")).place(x=20,y=20)
        self.label_sort_by_col = Label(self.window,text='Key words',font=("Arial", 15)).place(x=150,y=25)
        self.label_sort_order = Label(self.window,text="Methods",font=("Arial", 15)).place(x=550,y=25)
        
        self.comboxlist_by_col = ttk.Combobox(self.window,textvariable=StringVar(),font=("Arial", 12),width = 20,height=20)
        self.comboxlist_by_col["values"]=("----choose type","Vehicle Type","DirectionTime_O","GantryID_O","DirectionTime_D",
                                          "GantryID_D","Trip Length","Trip End","Trip Information")
        self.comboxlist_by_col.current(0)
        self.comboxlist_by_col.pack()
        self.comboxlist_by_col.place(x=300,y=30)
        
        self.comboxlist_order = ttk.Combobox(self.window,textvariable=StringVar(),font=("Arial", 12),width = 20,height=20)
        self.comboxlist_order["values"]=('----choose method','Ascending', 'Descending')
        self.comboxlist_order.current(0)
        self.comboxlist_order.pack()
        self.comboxlist_order.place(x=650,y=30)
        
        self.gohome()
        
    def gohome_data(self, all_data):
        return all_data
    
    def gohome(self):
        """
        Function behind the home command.
        show the description of our system

        """
        global TEXT
        global df_now
        df_now = self.gohome_data(all_data)
        TEXT = Text(self.window, height = 700, width=1200, font=("Courier", 21, 'bold', "italic"))
        TEXT.insert(END,"\n\n\n\n\n")
        TEXT.insert(END,"\t\t Welcome to our system! \n\n")
        TEXT.insert(END,"\t\t This is an inquiry system for Taiwan traffic data. \n\n")
        TEXT.insert(END,"\t\t In the system, you can \n")
        TEXT.insert(END,"\t\t     1.Search various types of data\n")
        TEXT.insert(END,"\t\t     2.Sort the data in any way you like\n\n")
        TEXT.insert(END,"\t\t Now, let's start to traveling in the data!:\n")
        TEXT.configure(state='disabled')
        TEXT.pack()
        TEXT.place(x=5,y=150)
        
    def Reset(self):
        """
        Function behind the reset command.
        show the original data

        """
        global df_now
        global TEXT
        if TEXT!=1:
            TEXT.destroy()
            TEXT = 1
        frame = Frame(self.window, height = 700, width = 1300)
        frame.pack(fill='both', expand=True)
        frame.place(x=5,y=150)
        df_now = self.gohome_data(all_data)
        pt = Table(frame, dataframe = df_now, height = 700,width = 1200)
        pt.show()
        pt.rowheader.maxwidth = 60
        pt.redraw()
        pass
    
    def sort(self):
        """
        Function behind the sort command.

        - keywords: the part the user sort by.
        - sortway: sort method we choose
        - pt: the final result we will show on the window
        """
        global TEXT
        if TEXT!=1:
            TEXT.destroy()
            TEXT = 1
        keywords = self.comboxlist_by_col.get() ### get the current keywords the user want to search
        sortway = self.comboxlist_order.get() ### get the current sort way the user want to use
        if sortway =="----choose method":
            mbox.showerror('Python Error Message','Error: You show first choose the sorting way!')  
            return
        if keywords =="----choose type":
            mbox.showerror('Python Error Message','Error: You show first choose the key word!')  
            return  
        global df_now
        frame = Frame(self.window, height = 700, width = 1200)
        frame.pack(fill='both', expand=True)
        frame.place(x=5,y=150)
        sort_ = s.my_sort(df_now, keywords, sortway)    
        pt = Table(frame, dataframe=sort_,height = 700,width = 1200)
        pt.show()
        pt.rowheader.maxwidth = 60
        pt.redraw()
        pass

In [18]:
### get the original data from in pandas format
cols = ['Vehicle Type','DirectionTime_O','GantryID_O',
        'DirectionTime_D','GantryID_D','Trip Length',
        'Trip End','Trip Information']
df = pd.read_csv("TDCS_M06A_20190830_080000.csv", names = cols)

all_data = df
s = Server()
i = interface(data = df)
i.init_interface()

def search():
    """
    Function behind the search command.

    - search_by_col: the part the user sort by.
    - text: contents we want to search
    - pt: the final result we will show on the window
    """
    global TEXT
    if TEXT!=1:
        TEXT.destroy()
        TEXT = 1
    search_by_col = comboxlist_search_col.get() ### get the current keywords the user want to search
    if search_by_col == "Trip Length":
        text = [search_text1.get(),search_text2.get()]
    elif "DirectionTime" in search_by_col:
        text = [search_text3.get(),search_text4.get(),
               search_text5.get(),search_text6.get(),
               search_text7.get(),search_text8.get(),]
    else:
        text = search_text.get() ### get the contents users want to search
    if search_by_col =="----choose type":
        mbox.showerror('Python Error Message','Error: You show first choose the key word!')  
        return
        
    search_ = s.my_search(df, search_by_col, text)
    global df_now
    frame = Frame(i.window,height = 700, width = 1200)
    frame.pack(fill='both', expand=True)
    frame.place(x=5,y=150)
    df_now = search_
    pt = Table(frame, dataframe=search_,height = 700,width = 1200)
    pt.show()
    pt.rowheader.maxwidth = 60
    pt.redraw()
    pass

def delete(elemlist):
    for e in elemlist:
        e.destroy()

def callback(event):
    typ = comvalue_search_col.get()
    
    global search_text,lable_contents,lable_contents1,lable_contents2,search_text1,search_text2
    global lable_contents3,lable_contents4,search_text3,search_text4,search_text5,search_text6,search_text7,search_text8
    contlist = [search_text,lable_contents]
    lenglist = [lable_contents1,lable_contents2,search_text1,search_text2]
    timelist = [lable_contents3,lable_contents4,search_text3,
                search_text4,search_text5,search_text6,search_text7,search_text8]
    if typ == "Trip Length":
        if lable_contents:     
            delete(contlist)
            search_text = 0
            lable_contents = 0

        if lable_contents3:
            delete(timelist)
            lable_contents3=0
            lable_contents4=0
            search_text3=0
            search_text4=0
            search_text5=0
            search_text6=0
            search_text7=0
            search_text8=0
        if not lable_contents1:
            lable_contents1=Label(i.window,text="Between",font=("Arial", 15))
            lable_contents1.place(x=550,y=85)
            lable_contents2=Label(i.window,text="and",font=("Arial", 15))
            lable_contents2.place(x=550+175,y=85)
            search_text1 = Entry(i.window,width = 5,font=("Arial", 12))
            search_text1.place(x=650,y=90)
            search_text2 = Entry(i.window,width = 5,font=("Arial", 12))
            search_text2.place(x=650+125,y=90)
    elif "DirectionTime_D" == typ or "DirectionTime_O" == typ:
        if lable_contents1:
            delete(lenglist)
            lable_contents1=0
            lable_contents2=0
            search_text1=0
            search_text2=0
        if lable_contents:
            delete(contlist)
            search_text = 0
            lable_contents = 0
        if not lable_contents3:
            lable_contents3=Label(i.window,text="From 2019/08/30",font=("Arial", 15))
            lable_contents3.place(x=550,y=70)
            lable_contents4=Label(i.window,text="To     2019/08/30",font=("Arial", 15))
            lable_contents4.place(x=550,y=100)
            search_text3 = entry(i.window,placeholder = '(hh)',width = 4,font=("Arial", 12))
            search_text3.place(x=720,y=75)
            search_text4 = entry(i.window,placeholder = '(mm)',width = 4,font=("Arial", 12))
            search_text4.place(x=770,y=75)
            search_text5 = entry(i.window,placeholder = '(ss)',width = 4,font=("Arial", 12))
            search_text5.place(x=820,y=75)
            search_text6 = entry(i.window,placeholder = '(hh)',width = 4,font=("Arial", 12))
            search_text6.place(x=720,y=105)
            search_text7 = entry(i.window,placeholder = '(mm)',width = 4,font=("Arial", 12))
            search_text7.place(x=770,y=105)
            search_text8 = entry(i.window,placeholder = '(ss)',width = 4,font=("Arial", 12))
            search_text8.place(x=820,y=105)
    else: 
        if lable_contents1!=0:
            delete(lenglist)
            lable_contents1=0
            lable_contents2=0
            search_text1=0
            search_text2=0
        if lable_contents3!=0:
            delete(timelist)
            lable_contents3=0
            lable_contents4=0
            search_text3=0
            search_text4=0
            search_text5=0
            search_text6=0
            search_text7=0
            search_text8=0
        if not lable_contents:
            lable_contents = Label(i.window,text="Contents",font=("Arial", 15))
            lable_contents.place(x=550,y=85)
            search_text = Entry(i.window,width = 22,font=("Arial", 12))
            search_text.place(x=650,y=90)

###define the labels for searching part
Label(i.window,text="Search:",font=("Courier", 20, "bold", "italic")).place(x=20,y=80)
Label(i.window,text='Key words',font=("Arial", 15)).place(x=150,y=85)
lable_contents = Label(i.window,text="Contents",font=("Arial", 15))
lable_contents.place(x=550,y=85)

# define attribute we set to search
lable_contents1=0
lable_contents2=0
search_text1=0
search_text2=0
lable_contents3=0
lable_contents4=0
search_text3=0
search_text4=0
search_text5=0
search_text6=0
search_text7=0
search_text8=0
search_text = Entry(i.window,width = 22,font=("Arial", 12))
search_text.place(x=650,y=90)
comvalue_search_col = StringVar()
comboxlist_search_col=ttk.Combobox(i.window,textvariable=comvalue_search_col,font=("Arial", 12),width = 20,height=20)
comboxlist_search_col["values"]=("----choose type","Vehicle Type","DirectionTime_O","GantryID_O","DirectionTime_D",
                      "GantryID_D","Trip Length","Trip End","Trip Information")
comboxlist_search_col.current(0)

comboxlist_search_col.bind('<<ComboboxSelected>>', callback)
comboxlist_search_col.pack()
comboxlist_search_col.place(x=300,y=90)
button_search = Button(i.window,text="Search",font=("Arial", 15),command=search,width=15,height = 1).place(x=900,y=80)

i.window.mainloop()